In [1]:
# 0. generate random order of edges
# 1. so that it actually starts with building a tree
# 2. build EPPDC for this tree
# now we iteratively add an edge (a,b)
# 4. bruteforce one direction as in the usual PPDC
# 5. bruteforce all sequences Q = b v1 v2 ... vt \inf
# 6. select the sequence which would give an EPPDC
# ...
# 7. profit! (TODO: prove that process never fails)

from collections import defaultdict
from copy import deepcopy
from enum import Enum
import random
import time

from unionfind import unionfind

In [2]:
# stuff

def gen_edges(n):
    edges = []
    for i in range(n):
        for j in range(i + 1, n):
            edges.append((i, j))
    random.shuffle(edges)
    return edges

def sort_edges_to_build_a_tree(edges, n, verbose=False):
    u = unionfind(n)
    tree_edges = []
    non_tree_edges = []
    for edge in edges:
        v1, v2 = edge
        if u.issame(v1, v2):
            non_tree_edges.append((v1, v2))
        else:
            u.unite(v1, v2)
            tree_edges.append(edge)
    random.shuffle(non_tree_edges)
    if verbose:
        print(f'{tree_edges=}')
    assert (len(tree_edges) == n - 1)
    return tree_edges, non_tree_edges


class Graph():
    def __init__(self, n, edges):
        self.n = n
        self.edges = edges
        self.neibs = dict()
        for v in range(n):
            self.neibs[v] = []
        for v1, v2 in edges:
            self.neibs[v1].append(v2)
            self.neibs[v2].append(v1)


class EdgePath():
    def __init__(self):
        self.edges = []
        self._vertices_set = set()
        self._start = None
        self._finish = None
        self.ends = set([self._start, self._finish])

    def _reverse(self):
        self.edges.reverse()
        for edge_idx, edge in enumerate(self.edges):
            self.edges[edge_idx] = (edge[1], edge[0])
        self._start, self._finish = self._finish, self._start

    def _update_start(self, vertex):
        self._start = vertex
        self.ends = set([self._start, self._finish])

    def _update_finish(self, vertex):
        self._finish = vertex
        self.ends = set([self._start, self._finish])

    def append(self, edge):
        if self.edges:
            assert edge[1] not in self._vertices_set
            if self._start == edge[0]:
                self._reverse()
            assert self._finish == edge[0]
        self.edges.append(edge)
        for vertex in edge:
            self._vertices_set.add(vertex)
        if self._start is None:
            self._start = self.edges[0][0]
        self._update_finish(edge[-1])

    def shrink(self):
        assert self.edges is not None
        assert len(self.edges) > 0
        self._vertices_set.remove(self._finish)
        self.edges.pop()
        assert len(self.edges) > 0
        self._update_finish(self.edges[-1][-1])

    def rebuild(self, new_vertices):
        self.edges = []
        self._vertices_set = set()
        self._start = None
        self._finish = None
        self.ends = set([self._start, self._finish])
        for i in range(len(new_vertices) - 1):
            self.append((new_vertices[i], new_vertices[i + 1]))

    def __contains__(self, vertex):
        return vertex in self._vertices_set

    def get_vertices(self):
        vertices = []
        for edge in self.edges:
            vertices.append(edge[0])
        vertices.append(self._finish)
        return vertices

    def __repr__(self):
        return str(self.get_vertices())


class EPPDC():
    def __init__(self, n):
        self.n = n
        self.paths = []

    def append(self, path):
        self.paths.append(path)

    def is_connected(self):
        u = unionfind(n)
        for path in self.paths:
            u.unite(*path.ends)
        return len(u.groups()) == 1

    def is_eppdc(self):
        if not self.is_connected():
            return False
        end_counts = defaultdict(int)
        edge_counts = defaultdict(int)
        for path in self.paths:
            for end in path.ends:
                end_counts[end] += 1
            for edge in path.edges:
                sorted_edge = edge
                if sorted_edge[0] > sorted_edge[1]:
                    sorted_edge = (edge[1], edge[0])
                edge_counts[sorted_edge] += 1
        for end, count in end_counts.items():
            if count != 2:
                print(f'fail in END counts; {end=}, {count=}')
                for path in self.paths:
                    print(f'{path=}')
                return False
        for edge, count in edge_counts.items():
            if count != 2:
                print(f'fail in EDGE counts; {edge=}, {count=}')
                return False
        return True

    def __iter__(self):
        return self.paths.__iter__()

    def __next__(self):
        return self.paths.__next__()


class DFSState():
    def __init__(self, n):
        self.visited = set()
        self.eppdc = EPPDC(n)
        self.cur_path = EdgePath()

    def dfs(self, v1, graph):
        self.visited.add(v1)

        # traversing children
        for v2 in graph.neibs[v1]:
            if v2 in self.visited:
                continue
            self.cur_path.append((v1, v2))
            self.dfs(v2, graph)
            self.cur_path.append((v2, v1))

        # leaving vertex v1
        self.eppdc.append(self.cur_path)
        self.cur_path = EdgePath()

    def knuth(self, v1, graph, mode='bfs'):
        assert mode in ['bfs', 'dfs']
        next_mode = 'dfs'
        if mode == 'dfs':
            next_mode = 'bfs'

        self.visited.add(v1)

        if mode == 'bfs':
            if v1 != 0:
                self.eppdc.append(self.cur_path)
                self.cur_path = EdgePath()

        # traversing children
        for v2 in graph.neibs[v1]:
            if v2 in self.visited:
                continue
            self.cur_path.append((v1, v2))
            self.knuth(v2, graph, next_mode)
            self.cur_path.append((v2, v1))

        # leaving vertex v1
        if mode == 'dfs' or v1 == 0:
            self.eppdc.append(self.cur_path)
            self.cur_path = EdgePath()


def build_tree_eppdc(graph, n):
    # idea: do some kind of dsf-like tree traversal: bfs, dfs, or Knuth
    # for simplicity we go with dfs
    # also: here we introduce the data structures, useful for describing EPPDC

    # algorithm: start at root, root will be 1 end of path
    # go 1 step deeper, add edge to path
    # go further, until the leaf
    # leaf will be a second end
    # start a new path at the leaf
    # continue in this manner
    # actual algorithm is: we start a new path each time we need to leave the vertex
    state = DFSState(n)
    state.dfs(0, graph)
    # TODO: remove, or allow to switch with parameter
#     state.knuth(0, graph, 'bfs')
    return state.eppdc

class State(Enum):
    SUCCESS = 0
    FAIL = 1
    BACKTRACK = 2

In [ ]:
# THE ALGORITHM ITSELF
def extend_with_backtrack(n, v1, v2, eppdc, part,
                          use_backtrack=False, verbose=False):
    # we want to add edge (v1, v2),
    # extend some path ending at v1 to v2

    if part == 1:
        assert not use_backtrack

    original_v1 = v1
    checked_v1s = set()  # checked_v1s

    # FIXME: remove
#     iteration_count = 0
#     upper_bound = n ** 3

    while True:
#         iteration_count += 1
#         if iteration_count >= upper_bound:
#             return (State.FAIL, (v1, v2))

        # search the paths, ending with v1
        v1_paths = []
        for possible_v1_path in eppdc:
            if v1 in possible_v1_path.ends:
                v1_paths.append(possible_v1_path)
        random.shuffle(v1_paths)

        switched = False
        for path in v1_paths:
            if v2 not in path:
                path.append((v1, v2))
                if eppdc.is_connected():
                    if part == 2:
                        if not eppdc.is_eppdc():
                            print('BUG')
                    return (State.SUCCESS, (v1, v2))
                else:
                    path.shrink()
                    if verbose:
                        print(f'  part{part}.x: {v2} not in path {path}, but not eppdc')
            else:
                # toy example
                # v1, v2 = (4, 3)
                # path = [2, 3, 1, 4]
                # v2 is in path
                # => new path:
                # [2, 3, 4, 1]
                # new v1, v2 = (1, 3)

                path_vertices = path.get_vertices()
                if path_vertices[0] == v1:
                    path_vertices.reverse()
                if path_vertices[-2] == v2:
                    # try to backtrack

                    # FIXME: analyze or simplify
                    if (v1 != original_v1) and use_backtrack and len(path_vertices) >= 3:
                        assert (part == 2)
                        if part == 2:
                            if verbose:
                                print(f'  part{part}.x: try to backtrack')
                            new_path_vertices = path_vertices[:-1]
                            if verbose:
                                print(f'  before backtrack rebuild')
                                for debug_path in eppdc.paths:
                                    print(f'    {debug_path=}')
                            path.rebuild(new_path_vertices)
                            if not eppdc.is_eppdc():
                                if verbose:
                                    print('backtrack failed')
                                path.rebuild(path_vertices)
                                continue
                            else:
                                assert eppdc.is_eppdc()
                                switched = True
                                if verbose:
                                    print(path_vertices, '=>', new_path_vertices)
                                    print(f'  after backtrack rebuild')
                                    for debug_path in eppdc.paths:
                                        print(f'    {debug_path=}')
                                return (State.BACKTRACK, (v1, v2))
                        else:
                            if verbose:
                                print('FIXME_FOR_PART1')
                            continue
                    else:
                        if v1 != original_v1:
                            if verbose:
                                print(f'  part{part}.x: could try to backtrack edge ({v1}, {v2})')
                        continue

                new_path_vertices = []
                for v in path_vertices:
                    new_path_vertices.append(v)
                    if v == v2:
                        break
                for v in reversed(path_vertices):
                    if v == v2:
                        break
                    new_path_vertices.append(v)

                if verbose:
                    print('before rebuild')

                new_v1 = new_path_vertices[-1]

                # checked_v1s
                if (new_v1, v1) in checked_v1s:
                    continue
                checked_v1s.add((v1, new_v1))

                assert eppdc.is_connected()
                path.rebuild(new_path_vertices)
                if not eppdc.is_connected():
                    path.rebuild(path_vertices)
                    assert eppdc.is_connected()
                    continue

                if verbose:
                    print('after rebuild')
                    print(path_vertices, '=>', new_path_vertices)
                    print(f'  current paths')
                    for debug_path in eppdc.paths:
                        print(f'{debug_path=}')

                switched = True
                v1 = new_v1
                break

        if switched:
            if verbose:
                print(f'  part{part}.x: adding edge ({v1}, {v2})')
        else:
            break

    return (State.FAIL, (v1, v2))  # checked_v1s
#     assert False

In [ ]:
# n = 5
n = 6
# n = 7
# n = 8
# n = 9
# n = 10
# n = 11
# n = 12
# n = 13
# n = 14
# n = 15
# n = 20

# verbose = True
verbose = False

print_stats = True
# print_stats = False

backtrack_iteration = 4

max_rep = 10000
# max_rep = 1000
# max_rep = 100
# max_rep = 1

for rep0 in range(max_rep):
    if (rep0 % 100 == 0):
#     if (rep0 % 10 == 0):
        print(rep0, '/', max_rep)
#     n = random.randint(5, 25)
    seed = random.randint(0, 1000000000)
    
    # FIXME (debug)
#     verbose = True
#     n = 8; seed = 217499430; rep0 = 4645  # needs v1, v2 = v2, v1
#     n = 8; seed = 835385195; rep0 = 799  # needs backtrack
    
    random.seed(seed)

    # 0. generate random order of edges
    edges = gen_edges(n)
    # 1. so that we actually start with building a tree
    tree_edges, non_tree_edges = sort_edges_to_build_a_tree(edges, n, verbose=verbose)
    
    random.seed(seed + rep0)

    # 2. build EPPDC for this tree
    tree = Graph(n=n, edges=tree_edges)
    tree_eppdc = build_tree_eppdc(tree, n)
    assert tree_eppdc.is_eppdc()
    if verbose:
        for path in tree_eppdc:
            print(path)

    # now we iteratively add an edge
    # steps 4.-6.
    for rep1 in range(1):
        eppdc = deepcopy(tree_eppdc)

        for edge_idx, (orig_v1, orig_v2) in enumerate(non_tree_edges):
            # FIXME (debug)
#             if edge_idx == 20:
#                 verbose = True
            
            eppdc_before_part1 = deepcopy(eppdc)

            v1, v2 = orig_v1, orig_v2
            if random.random() > 0.5:
                v1, v2 = v2, v1

            # FIXME: remove + configure properly
            iteration_count = 0
            use_backtrack = False

            while True:
                success1 = State.FAIL
                success2 = State.FAIL

                iteration_count += 1
                if iteration_count > backtrack_iteration:
                    use_backtrack = True
#                 if iteration_count > 200:
#                     break

                v1, v2 = v2, v1  # looks like we need this
                eppdc = deepcopy(eppdc_before_part1)

                if verbose:
                    print(f'{edge_idx=} before part 1; adding edge {(v1, v2)}')
                    for path in eppdc:
                        print(path)

                success1, _ = extend_with_backtrack(n, v1, v2, eppdc, 1,
                                                    verbose=verbose)

                if verbose:
                    print(f'{edge_idx=} part1 success: {success1}')
                    if success1 == State.SUCCESS:
                        print(f'{edge_idx=} after part 1;')
                        for path in eppdc:
                            print(path)
                        print(f'adding edge {(v2, v1)}')

                if success1 == State.SUCCESS:
                    success2, (new_v2, new_v1) = extend_with_backtrack(n, v2, v1, eppdc, 2,
                                                                       use_backtrack=use_backtrack,
                                                                       verbose=verbose)
                else:
                    success2 = State.FAIL

                if verbose:
                    print(f'{edge_idx=} part2 success: {success2}')    
                    if success2 == State.SUCCESS:
                        print(f'{edge_idx=} after part 2')
                        for path in eppdc:
                            print(path)
                    time.sleep(0.1)

                if success2 != State.FAIL:
                    assert eppdc.is_connected()
                    assert eppdc.is_eppdc()

                if success2 == State.SUCCESS:
                    # assert not use_backtrack  # FIXME: ideally this should be the case
                    break

                if success2 == State.BACKTRACK:
                    if verbose:
                        print(f'backtrack edge ({v1}, {v2}); now try ({new_v1}, {new_v2})')
                        print('')

                    v1, v2 = new_v1, new_v2
                    eppdc_before_part1 = deepcopy(eppdc)
                    iteration_count = 0
                    use_backtrack = False
                    continue

                if verbose:
                    print('fail, try again')
                    print('')

            if success2 != State.SUCCESS:
                print(f'{n=}; {seed=}; {rep0=}')
                print(f'{success1=}, {success2=}, {edge_idx=} vs {len(non_tree_edges)}')
                print(f'(v1, v2) = ({v1}, {v2})')
                for path in eppdc_before_part1:
                    print(path)
                assert False
            assert eppdc.is_eppdc()

print('DONE')

In [ ]:
# THE ALGORITHM ITSELF
def extend_with_backtrack(n, v1, v2, eppdc, part,
                          use_backtrack=False,
                          start_from_idx=0,
                          verbose=False):
    # we want to add edge (v1, v2),
    # extend some path ending at v1 to v2

    if part == 1:
        assert not use_backtrack

    original_v1 = v1
    checked_switches = set()

    while True:
        # search the paths, ending with v1
        v1_paths = []
        for possible_v1_path in eppdc:
            if v1 in possible_v1_path.ends:
                v1_paths.append(possible_v1_path)
        if len(checked_switches) == 0:  # start_from_idx
            assert len(v1_paths) > start_from_idx
            v1_paths = v1_paths[start_from_idx:]
        else:
#             if use_backtrack:
            random.shuffle(v1_paths)  # FIXME: remove randomness

        switched = False
        for path in v1_paths:
            if v2 not in path:
                path.append((v1, v2))
                if eppdc.is_connected():
                    if part == 2:
                        if not eppdc.is_eppdc():
                            print('BUG')
                    return (State.SUCCESS, (v1, v2))
                else:
                    path.shrink()
                    if verbose:
                        print(f'  part{part}.x: {v2} not in path {path}, but not eppdc')
            else:
                # toy example
                # v1, v2 = (4, 3)
                # path = [2, 3, 1, 4]
                # v2 is in path
                # => new path:
                # [2, 3, 4, 1]
                # new v1, v2 = (1, 3)

                path_vertices = path.get_vertices()
                if path_vertices[0] == v1:
                    path_vertices.reverse()
                if path_vertices[-2] == v2:
                    # try to backtrack

                    # FIXME: analyze or simplify
                    if (v1 != original_v1) and use_backtrack and len(path_vertices) >= 3:
                        assert (part == 2)
                        if part == 2:
                            if verbose:
                                print(f'  part{part}.x: try to backtrack')
                            new_path_vertices = path_vertices[:-1]
                            if verbose:
                                print(f'  before backtrack rebuild')
                                for debug_path in eppdc.paths:
                                    print(f'    {debug_path=}')
                            path.rebuild(new_path_vertices)
                            if not eppdc.is_eppdc():
                                if verbose:
                                    print('backtrack failed')
                                path.rebuild(path_vertices)
                                continue
                            else:
                                assert eppdc.is_eppdc()
                                switched = True
                                if verbose:
                                    print(path_vertices, '=>', new_path_vertices)
                                    print(f'  after backtrack rebuild')
                                    for debug_path in eppdc.paths:
                                        print(f'    {debug_path=}')
                                return (State.BACKTRACK, (v1, v2))
                        else:
                            if verbose:
                                print('FIXME_FOR_PART1')
                            continue
                    else:
                        if v1 != original_v1:
                            if verbose:
                                print(f'  part{part}.x: could try to backtrack edge ({v1}, {v2})')
                        continue

                new_path_vertices = []
                for v in path_vertices:
                    new_path_vertices.append(v)
                    if v == v2:
                        break
                for v in reversed(path_vertices):
                    if v == v2:
                        break
                    new_path_vertices.append(v)

                if verbose:
                    print('before rebuild')

                new_v1 = new_path_vertices[-1]

#                 if not use_backtrack and ((new_v1, v1) in checked_switches):
#                     continue
                checked_switches.add((v1, new_v1))

                assert eppdc.is_connected()
                path.rebuild(new_path_vertices)
                if not eppdc.is_connected():
                    path.rebuild(path_vertices)
                    assert eppdc.is_connected()
                    continue

                if verbose:
                    print('after rebuild')
                    print(path_vertices, '=>', new_path_vertices)
                    print(f'  current paths')
                    for debug_path in eppdc.paths:
                        print(f'{debug_path=}')

                switched = True
                v1 = new_v1
                break

        if switched:
            if verbose:
                print(f'  part{part}.x: adding edge ({v1}, {v2})')
        else:
            break

    return (State.FAIL, (v1, v2))

In [ ]:
idx1 = [0, 1, 0, 1, 0, 1]
idx2 = [0, 0, 1, 1, 2, 2]

# n = 5
n = 6
# n = 7
# n = 8
# n = 9
# n = 10
# n = 11
# n = 12
# n = 13
# n = 14
# n = 15
# n = 20

# verbose = True
verbose = False

print_stats = True
# print_stats = False

max_rep = 10000
# max_rep = 1000
# max_rep = 100
# max_rep = 1

for rep0 in range(max_rep):
    if (rep0 % 100 == 0):
#     if (rep0 % 10 == 0):
        print(rep0, '/', max_rep)
#     n = random.randint(5, 25)
    seed = random.randint(0, 1000000000)
    
    # FIXME (debug)
#     verbose = True
#     n = 8; seed = 217499430; rep0 = 4645  # needs v1, v2 = v2, v1
#     n = 8; seed = 835385195; rep0 = 799  # needs backtrack
    
    random.seed(seed)

    # 0. generate random order of edges
    edges = gen_edges(n)
    # 1. so that we actually start with building a tree
    tree_edges, non_tree_edges = sort_edges_to_build_a_tree(edges, n, verbose=verbose)

    # 2. build EPPDC for this tree
    tree = Graph(n=n, edges=tree_edges)
    tree_eppdc = build_tree_eppdc(tree, n)
    assert tree_eppdc.is_eppdc()
    if verbose:
        for path in tree_eppdc:
            print(path)

    # now we iteratively add an edge
    # steps 4.-6.

    eppdc = deepcopy(tree_eppdc)

    for edge_idx, (orig_v1, orig_v2) in enumerate(non_tree_edges):
        # FIXME (debug)
#         if edge_idx == 20:
#             verbose = True

        eppdc_before_part1 = deepcopy(eppdc)
        v1, v2 = orig_v1, orig_v2
        # FIXME: i think we should keep a list of backtracked edges,
        # so we don't repeat them infinitely
        backtracked = True
        successed = False

        while not successed and backtracked:
            backtracked = False

            for use_backtrack in [False, True]:
                if successed or backtracked:
                    break

                for v1v2rep in range(2):
                    if successed or backtracked:
                        break

                    v1, v2 = v2, v1  # feels like some hack, but looks like we need this

                    for idx_pair in zip(idx1, idx2):
                        if successed or backtracked:
                            break

                        eppdc = deepcopy(eppdc_before_part1)

                        if verbose:
                            print(f'{edge_idx=} before part 1; adding edge {(v1, v2)}')
                            for path in eppdc:
                                print(path)

                        success1, _ = extend_with_backtrack(n, v1, v2, eppdc, 1,
                                                            start_from_idx=idx_pair[0],
                                                            verbose=verbose)

                        if verbose:
                            print(f'{edge_idx=} part1 success: {success1}')
                            if success1 == State.SUCCESS:
                                print(f'{edge_idx=} after part 1;')
                                for path in eppdc:
                                    print(path)
                                print(f'adding edge {(v2, v1)}')

                        if success1 == State.SUCCESS:
                            success2, (new_v1, new_v2) = extend_with_backtrack(n, v2, v1, eppdc, 2,
                                                                               use_backtrack=use_backtrack,
                                                                               start_from_idx=idx_pair[1],
                                                                               verbose=verbose)
                        else:
                            success2 = State.FAIL

                        if verbose:
                            print(f'{edge_idx=} part2 success: {success2}')    
                            if success2 == State.SUCCESS:
                                print(f'{edge_idx=} after part 2')
                                for path in eppdc:
                                    print(path)
                            time.sleep(0.1)

                        if use_backtrack:
                            assert success2 != State.SUCCESS

                        if success2 != State.FAIL:
                            assert eppdc.is_connected()
                            assert eppdc.is_eppdc()

                            if success2 == State.SUCCESS:
                                successed = True
                            else:
                                assert (success2 == State.BACKTRACK)
                                if verbose:
                                    print(f'{edge_idx=} vs {len(non_tree_edges)}')
                                    print(f'backtrack edge ({v1}, {v2}); now try ({new_v1}, {new_v2})')
                                    print('')

                                backtracked = True
                                v1, v2 = new_v1, new_v2
                                eppdc_before_part1 = deepcopy(eppdc)

                            break

                        if verbose:
                            print('fail, try again')
                            print('')

        if not successed:
            print(f'{n=}; {seed=}; {rep0=}')
            print(f'{edge_idx=} vs {len(non_tree_edges)}')
            print(f'(v1, v2) = ({v1}, {v2})')
            for path in eppdc_before_part1:
                print(path)
            assert False

        assert eppdc.is_eppdc()

print('DONE')

In [97]:
# THE ALGORITHM ITSELF
def extend_with_backtrack(n, v1, v2, eppdc, part,
                          use_backtrack=False, verbose=False):
    # we want to add edge (v1, v2),
    # extend some path ending at v1 to v2

#     do_backtrack = False
    do_backtrack = use_backtrack
    original_v1 = v1

    # FIXME: remove
    iteration_count = 0
    upper_bound = n ** 3

    while True:
        iteration_count += 1
        if iteration_count >= upper_bound:
            return (State.FAIL, (v1, v2))

        if part == 1:
            do_backtrack = False

        # search the paths, ending with v1
        v1_paths = []
        for possible_v1_path in eppdc:
            if v1 in possible_v1_path.ends:
                v1_paths.append(possible_v1_path)
        random.shuffle(v1_paths)

        switched = False
        for path in v1_paths:
            if v2 not in path:
                path.append((v1, v2))
#                 if eppdc.is_connected():
                if part == 1 or eppdc.is_connected():  # part1_disconnected
                    if part == 2:
                        if not eppdc.is_eppdc():
                            print('BUG')
                    return (State.SUCCESS, (v1, v2))
                else:
                    path.shrink()
                    if verbose:
                        print(f'  part{part}.x: {v2} not in path {path}, but not eppdc')
            else:
                # toy example
                # v1, v2 = (4, 3)
                # path = [2, 3, 1, 4]
                # v2 is in path
                # => new path:
                # [2, 3, 4, 1]
                # new v1, v2 = (1, 3)

                path_vertices = path.get_vertices()
                if path_vertices[0] == v1:
                    path_vertices.reverse()
                if path_vertices[-2] == v2:
                    # try to backtrack

                    if (v1 != original_v1) and do_backtrack and len(path_vertices) >= 3:  # FIXME
                        assert (part == 2)
                        if part == 2:
                            if verbose:
                                print(f'  part{part}.x: try to backtrack')
                            new_path_vertices = path_vertices[:-1]
                            if verbose:
                                print(f'  before backtrack rebuild')
                                for debug_path in eppdc.paths:
                                    print(f'    {debug_path=}')
                            path.rebuild(new_path_vertices)
                            if not eppdc.is_eppdc():
                                if verbose:
                                    print('backtrack failed')
                                path.rebuild(path_vertices)
                                continue
                            else:
                                assert eppdc.is_eppdc()
                                switched = True
                                if verbose:
                                    print(path_vertices, '=>', new_path_vertices)
                                    print(f'  after backtrack rebuild')
                                    for debug_path in eppdc.paths:
                                        print(f'    {debug_path=}')
                                return (State.BACKTRACK, (v1, v2))
                        else:
                            if verbose:
                                print('FIXME_FOR_PART1')
                            continue
                    else:
                        if v1 != original_v1:
                            if verbose:
                                print(f'  part{part}.x: could try to backtrack edge ({v1}, {v2})')
                        continue

                new_path_vertices = []
                for v in path_vertices:
                    new_path_vertices.append(v)
                    if v == v2:
                        break
                for v in reversed(path_vertices):
                    if v == v2:
                        break
                    new_path_vertices.append(v)

                if verbose:
                    print('before rebuild')

#                 assert eppdc.is_connected()  # FIXME: part1_disconnected
                path.rebuild(new_path_vertices)
#                 if not eppdc.is_connected():  # FIXME: part1_disconnected
#                 if not (part == 1 or eppdc.is_connected()):
#                     if verbose:
#                         print(f'  part{part}.x: not eppdc; tried {new_path_vertices}')
#                     path.rebuild(path_vertices)
#                     assert eppdc.is_connected()
#                     continue

                if verbose:
                    print('after rebuild')
                # FIXME: remove redundant assert
#                 assert eppdc.is_connected()
#                 assert (part == 1 or eppdc.is_connected())  # FIXME: part1_disconnected
                switched = True
                if verbose:
                    print(path_vertices, '=>', new_path_vertices)
                    print(f'  current paths')
                    for debug_path in eppdc.paths:
                        print(f'{debug_path=}')
                v1 = new_path_vertices[-1]
                break

        if switched:
            if verbose:
                print(f'  part{part}.x: adding edge ({v1}, {v2})')
        else:
            break

    return (State.FAIL, (v1, v2))  # FIXME: part1_disconnected
#     assert False

In [138]:
# n = 5
n = 6
# n = 7
# n = 8
# n = 9
# n = 10
# n = 11
# n = 12
# n = 13
# n = 14
# n = 15
# n = 20

# verbose = True
verbose = False

print_stats = True
# print_stats = False

max_rep = 10000
# max_rep = 100
# max_rep = 1

for rep0 in range(max_rep):
    if (rep0 % 100 == 0):
        print(rep0, '/', max_rep)
    seed = random.randint(0, 1000000000)
    
    # FIXME: debug
#     verbose = True
#     n=8; seed=217499430; rep0=4645  # needs v1, v2 = v2, v1
#     n=8; seed=835385195; rep0=799  # needs backtrack
    
    random.seed(seed)

    # 0. generate random order of edges
    edges = gen_edges(n)
    # 1. so that we actually start with building a tree
    tree_edges, non_tree_edges = sort_edges_to_build_a_tree(edges, n, verbose=verbose)
    
    random.seed(seed + rep0)

    # 2. build EPPDC for this tree
    tree = Graph(n=n, edges=tree_edges)
    tree_eppdc = build_tree_eppdc(tree, n)
    assert tree_eppdc.is_eppdc()
    if verbose:
        for path in tree_eppdc:
            print(path)

    # now we iteratively add an edge
    # steps 4.-6.
    for rep1 in range(1):
        eppdc = deepcopy(tree_eppdc)

        for edge_idx, (orig_v1, orig_v2) in enumerate(non_tree_edges):
            # FIXME: debug
#             if edge_idx == 20:
#                 verbose = True
            
            eppdc_before_part1 = deepcopy(eppdc)

            v1, v2 = orig_v1, orig_v2
            if random.random() > 0.5:
                v1, v2 = v2, v1

            # FIXME: remove
            iteration_count = 0
            # FIXME: configure properly
            use_backtrack = False

            while True:
                success1 = State.FAIL
                success2 = State.FAIL

                # FIXME: remove
                iteration_count += 1
                # FIXME: configure properly
                if iteration_count > 20:
                    use_backtrack = True
                if iteration_count > 200:
                    break

                v1, v2 = v2, v1  # looks like we need this
                eppdc = deepcopy(eppdc_before_part1)

                if verbose:
                    print(f'{edge_idx=} before part 1; adding edge {(v1, v2)}')
                    for path in eppdc:
                        print(path)

                success1, _ = extend_with_backtrack(n, v1, v2, eppdc, 1,
                                                    use_backtrack=False, verbose=verbose)

                if verbose:
                    print(f'{edge_idx=} part1 success: {success1}')
                    if success1 == State.SUCCESS:
                        print(f'{edge_idx=} after part 1;')
                        for path in eppdc:
                            print(path)
                        print(f'adding edge {(v2, v1)}')

                if success1 == State.SUCCESS:
                    success2, (new_v2, new_v1) = extend_with_backtrack(n, v2, v1, eppdc, 2,
                                                                       use_backtrack=use_backtrack, verbose=verbose)
                    if success2 != State.FAIL:
                        assert eppdc.is_connected()
                        assert eppdc.is_eppdc()

                if verbose:
                    print(f'{edge_idx=} part2 success: {success2}')    
                    if success2 == State.SUCCESS:
                        print(f'{edge_idx=} after part 2')
                        for path in eppdc:
                            print(path)
                    time.sleep(0.1)

                if success2 == State.SUCCESS:
                    break
                if success2 == State.BACKTRACK:
                    v1, v2 = new_v1, new_v2

                    eppdc_before_part1 = deepcopy(eppdc)
                    if verbose:
                        print('backtrack edge ({v1}, {v2}); now try ({new_v2}, {new_v1})')
                        print('')
                    continue

                if verbose:
                    print('fail, try again')
                    print('')

            if success2 != State.SUCCESS:
                print(f'{n=}; {seed=}; {rep0=}')
                print(f'{success1=}, {success2=}, {edge_idx=} vs {len(non_tree_edges)}')
                print(f'(v1, v2) = ({v1}, {v2})')
                for path in eppdc_before_part1:
                    print(path)
                assert False
            assert eppdc.is_eppdc()

print('DONE')

0 / 10000
100 / 10000
200 / 10000
300 / 10000
400 / 10000
500 / 10000
600 / 10000
700 / 10000
800 / 10000
900 / 10000
1000 / 10000
1100 / 10000
1200 / 10000
1300 / 10000
1400 / 10000
1500 / 10000
1600 / 10000
1700 / 10000
1800 / 10000
1900 / 10000
2000 / 10000
2100 / 10000
2200 / 10000
2300 / 10000
2400 / 10000
2500 / 10000
2600 / 10000
2700 / 10000
2800 / 10000
2900 / 10000
3000 / 10000
3100 / 10000
3200 / 10000
3300 / 10000
3400 / 10000
3500 / 10000
3600 / 10000
3700 / 10000
3800 / 10000
3900 / 10000
4000 / 10000
4100 / 10000
4200 / 10000
4300 / 10000
4400 / 10000
4500 / 10000
4600 / 10000
4700 / 10000
4800 / 10000
4900 / 10000
5000 / 10000
5100 / 10000
5200 / 10000
5300 / 10000
5400 / 10000
5500 / 10000
5600 / 10000
5700 / 10000
5800 / 10000
5900 / 10000
6000 / 10000
6100 / 10000
6200 / 10000
6300 / 10000
6400 / 10000
6500 / 10000
6600 / 10000
6700 / 10000
6800 / 10000
6900 / 10000
7000 / 10000
7100 / 10000
7200 / 10000
7300 / 10000
7400 / 10000
7500 / 10000
7600 / 10000
7700 / 1000

In [287]:
# THE ALGORITHM ITSELF
def extend(n, v1, v2, eppdc, part, verbose=False):
    # we want to add edge (v1, v2),
    # extend some path ending at v1 to v2
    # NOTE: "part" variable is only used for debug print purposes, code is same for part 1 and 2

    # NOTE: remove
    iteration_count = 0
    upper_bound = n ** 3

    while True:
        iteration_count += 1
        if iteration_count >= upper_bound:
            return False

        # search the paths, ending with v1
        v1_paths = []
        for possible_v1_path in eppdc:
            if v1 in possible_v1_path.ends:
                v1_paths.append(possible_v1_path)
        random.shuffle(v1_paths)

        switched = False
        for path in v1_paths:
            if v2 not in path:
                path.append((v1, v2))
                if eppdc.is_connected():
                    if part == 2:
                        if not eppdc.is_eppdc():
                            print('BUG')
                    return True
                else:
                    path.shrink()
                    if verbose:
                        print(f'  part{part}.x: {v2} not in path {path}, but not eppdc')
            else:
                # toy example
                # v1, v2 = (4, 3)
                # path = [2, 3, 1, 4]
                # v2 is in path
                # => new path:
                # [2, 3, 4, 1]
                # new v1, v2 = (1, 3)

                path_vertices = path.get_vertices()
                if path_vertices[0] == v1:
                    path_vertices.reverse()
                if path_vertices[-2] == v2:
                    continue

                new_path_vertices = []
                for v in path_vertices:
                    new_path_vertices.append(v)
                    if v == v2:
                        break
                for v in reversed(path_vertices):
                    if v == v2:
                        break
                    new_path_vertices.append(v)

                if verbose:
                    print('before rebuild')

                assert eppdc.is_connected()
                path.rebuild(new_path_vertices)
#                 if False: # not eppdc.is_connected():
                if not eppdc.is_connected():
                    if verbose:
                        print(f'  part{part}.x: not eppdc; tried {new_path_vertices}')
                    path.rebuild(path_vertices)
                    assert eppdc.is_connected()
                    continue

                if verbose:
                    print('after rebuild')
                assert eppdc.is_connected()
                switched = True
                if verbose:
                    print(path_vertices, '=>', new_path_vertices)
                    print(f'  current paths')
                    for debug_path in eppdc.paths:
                        print(f'{debug_path=}')
                v1 = new_path_vertices[-1]
                break

        if switched:
            if verbose:
                print(f'  part{part}.x: adding edge ({v1}, {v2})')
        else:
            break

    return False

In [290]:
# n = 5
n = 6
# n = 7
# n = 8
# n = 9
# n = 10
# n = 11
# n = 12
# n = 13
# n = 14
# n = 15
# n = 20

# verbose = True
verbose = False

print_stats = True
# print_stats = False

# max_rep = 1000
max_rep = 100
# max_rep = 1

for rep0 in range(max_rep):
    if (rep0 % 10 == 0):
        print(rep0, '/', max_rep)
    seed = random.randint(0, 1000000000)
    
    # NOTE: debug
#     verbose = True
    n=8; seed=434897521; rep0=45  # some time ago this example needed backtrack for the last edge
    
    random.seed(seed)

    # 0. generate random order of edges
    edges = gen_edges(n)
    # 1. so that we actually start with building a tree
    tree_edges, non_tree_edges = sort_edges_to_build_a_tree(edges, n, verbose=verbose)
    
    random.seed(seed + rep0)

    # 2. build EPPDC for this tree
    tree = Graph(n=n, edges=tree_edges)
    tree_eppdc = build_tree_eppdc(tree, n)
    assert tree_eppdc.is_eppdc()
    if verbose:
        for path in tree_eppdc:
            print(path)

    # now we iteratively add an edge
    # steps 4.-6.
    for rep1 in range(1):
        eppdc = deepcopy(tree_eppdc)

        for edge_idx, (orig_v1, orig_v2) in enumerate(non_tree_edges):
            # NOTE: debug
#             if edge_idx == 20:
#                 verbose = True
            
            eppdc_before_part1 = deepcopy(eppdc)

            v1, v2 = orig_v1, orig_v2
            if random.random() > 0.5:
                v1, v2 = v2, v1

            iteration_count = 0
            while True:
                success1 = False
                success2 = False

                iteration_count += 1
                if iteration_count > 20:
                    break
                v1, v2 = v2, v1
                eppdc = deepcopy(eppdc_before_part1)

                if verbose:
                    print(f'{edge_idx=} before part 1; adding edge {(v1, v2)}')
                    for path in eppdc:
                        print(path)

                success1 = extend(n, v1, v2, eppdc, 1, verbose=verbose)
#                 if not eppdc.is_connected():
#                     success1 = False

                if verbose:
                    print(f'{edge_idx=} part1 success: {success1}')
                    if success1:
                        print(f'{edge_idx=} after part 1;')
                        for path in eppdc:
                            print(path)
                        print(f'adding edge {(v2, v1)}')

                if success1:
                    success2 = extend(n, v2, v1, eppdc, 2, verbose=verbose)
#                     if not eppdc.is_connected():
#                         success2 = False

                if verbose:
                    print(f'{edge_idx=} part2 success: {success2}')    
                    if success2:
                        print(f'{edge_idx=} after part 2')
                        for path in eppdc:
                            print(path)
                    time.sleep(0.1)

                if success2:
                    break
                
                if verbose:
                    print('fail, try again')
                    print('')

            if not success2:
                print(f'{n=}; {seed=}; {rep0=}')
                print(f'{success1=}, {success2=}, {edge_idx=} vs {len(non_tree_edges)}')
                print(f'(v1, v2) = ({v1}, {v2})')
                for path in eppdc_before_part1:
                    print(path)
                assert False

            assert eppdc.is_eppdc()

print('DONE')

0 / 100
n=8; seed=434897521; rep0=45
success1=False, success2=False, edge_idx=20 vs 21
(v1, v2) = (5, 0)
[7, 0, 1, 5, 2, 6, 4, 3]
[1, 6, 2, 0, 4, 7, 3, 5]
[7, 5, 3, 1, 4, 2, 0, 6]
[2, 1, 7, 3, 6, 5, 4]
[3, 4, 7, 0, 6, 1, 2, 5]
[6, 3, 2, 4, 5, 7, 1, 0]
[1, 5, 6, 7, 2, 3, 0, 4]
[0, 3, 1, 4, 6, 7, 2]


AssertionError: 